### Manipulation of PeaskDB de novo-assisted database search results of Trocas 7 (high water, April 2019) incubation samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Exported at <1.0% FDR
    
Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 101) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

In [1]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/206_TROCAS7_Fusion_Apr2021_PEAKS_144/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/206_TROCAS7_Fusion_Apr2021_PEAKS_144


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls 

 206_TROCAS7_Fusion_Apr2021_PEAKS_144_peptide.csv
 206_TROCAS7_Fusion_Apr2021_PEAKS_144_protein.csv
 206_TROCAS7_Fusion_Apr2021_PEAKS_144_protein.fasta
'DB search psm.csv'
'de novo only peptides.csv'
 peptides.pep.xml
 protein-peptides.csv


In [4]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup206 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/206_TROCAS7_Fusion_Apr2021_PEAKS_144/206_TROCAS7_Fusion_Apr2021_PEAKS_144_peptide.csv")

# remove redundant rows
peaksdb206 = pd.DataFrame.drop_duplicates(peaksdbdup206)

print(peaksdb206.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source_File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb206.columns = columns

#remmove # spec and accession columns because they mess parsing up

del peaksdb206['#Spec']
del peaksdb206['Accession']
del peaksdb206['PTM']
del peaksdb206['AScore']

mean_length = peaksdb206['Length'].mean()
print(mean_length)

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup206))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb206))

#look at the dataframe
peaksdb206.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area Trocas7-206-Chav', 'Fraction', 'Scan', 'Source File', '#Spec',
       '#Spec Trocas7-206-Chav', 'Accession', 'PTM', 'AScore'],
      dtype='object')
12.01010101010101
# redundant peaksdb peptides in combined dataframe 99
# nonredundant peaksdb peptides in combined dataframe 99


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source_File
0,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,85.55,1881.8350,18,1.6,941.9263,90.66,2320000.0,26,15276,20210407_Trocas7_663_Chav206_DDA_120min_1.raw
1,LGEHNIDVLEGNEQFINAAK,71.87,2210.0967,20,5.4,737.7101,93.37,24800000.0,26,15772,20210407_Trocas7_663_Chav206_DDA_120min_1.raw
2,LGEHNIDVLEGNEQ(+.98)FINAAK,66.20,2211.0808,20,2.5,738.0361,95.43,322000.0,26,16149,20210407_Trocas7_663_Chav206_DDA_120min_1.raw
3,LGEHNIDVLEGNEQFIN(+.98)AAK,65.91,2211.0808,20,3.1,738.0365,94.81,259000.0,26,16034,20210407_Trocas7_663_Chav206_DDA_120min_1.raw
4,LGEHN(+.98)IDVLEGNEQFINAAK,64.45,2211.0808,20,2.5,738.0361,97.05,119000.0,26,16433,20210407_Trocas7_663_Chav206_DDA_120min_1.raw


In [6]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb206['A'] = peaksdb206['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb206['C'] = peaksdb206['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb206['D'] = peaksdb206['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb206['E'] = peaksdb206['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb206['F'] = peaksdb206['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb206['G'] = peaksdb206['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb206['H'] = peaksdb206['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb206 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb206['I'] = peaksdb206['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb206['K'] = peaksdb206['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb206['L'] = peaksdb206['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb206['M'] = peaksdb206['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb206['N'] = peaksdb206['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb206['P'] = peaksdb206['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb206['Q'] = peaksdb206['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb206['R'] = peaksdb206['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb206['S'] = peaksdb206['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb206['T'] = peaksdb206['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb206['V'] = peaksdb206['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb206['W'] = peaksdb206['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb206['Y'] = peaksdb206['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb206['c-carb'] = peaksdb206['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb206['m-oxid'] = peaksdb206['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb206['n-deam'] = peaksdb206['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a lamba function to enumerate the # of deamidated Q's in each peptide
peaksdb206['q-deam'] = peaksdb206['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb206['stripped_peptide'] = peaksdb206['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb206['stripped_length'] = peaksdb206['stripped_peptide'].apply(len)

peaksdb206['NAAF_num.'] = peaksdb206['Area'] / peaksdb206['stripped_length']

# total the number of modifications in sequence
peaksdb206['ptm-total'] = peaksdb206['c-carb'] + peaksdb206['m-oxid'] + peaksdb206['n-deam'] + peaksdb206['q-deam']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb206['stripped_IL']= peaksdb206['stripped_peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb206.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/206_CV_T00_GD_PDB/206_CV_T00_GD_PDB.csv")

# check out the results
peaksdb206.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,NAAF_num.,ptm-total,stripped_IL
0,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,85.55,1881.8350,18,1.6,941.9263,90.66,2320000.0,26,15276,...,0,2,0,0,0,SCLISGWGNTK,11,2.109091e+05,2,SCLLSGWGNTK
1,LGEHNIDVLEGNEQFINAAK,71.87,2210.0967,20,5.4,737.7101,93.37,24800000.0,26,15772,...,0,0,0,0,0,LGEHNIDVLEGNEQFINAAK,20,1.240000e+06,0,LGEHNLDVLEGNEQFLNAAK
2,LGEHNIDVLEGNEQ(+.98)FINAAK,66.20,2211.0808,20,2.5,738.0361,95.43,322000.0,26,16149,...,0,0,0,0,1,LGEHNIDVLEGNEQFINAAK,20,1.610000e+04,1,LGEHNLDVLEGNEQFLNAAK
3,LGEHNIDVLEGNEQFIN(+.98)AAK,65.91,2211.0808,20,3.1,738.0365,94.81,259000.0,26,16034,...,0,0,0,1,0,LGEHNIDVLEGNEQFINAAK,20,1.295000e+04,1,LGEHNLDVLEGNEQFLNAAK
4,LGEHN(+.98)IDVLEGNEQFINAAK,64.45,2211.0808,20,2.5,738.0361,97.05,119000.0,26,16433,...,0,0,0,1,0,LGEHNIDVLEGNEQFINAAK,20,5.950000e+03,1,LGEHNLDVLEGNEQFLNAAK


In [7]:
# keep only stripped peptide I/L and NAAF
db_206 = peaksdb206[['stripped_IL', 'Area', 'NAAF_num.']]

db_206.set_index('stripped_IL')

# write modified dataframe to new txt file
db_206.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/206_CV_T00_GD_PDB_ILnaafs.csv")

db_206.head()

,stripped_IL,Area,NAAF_num.
0,SCLLSGWGNTK,2320000.0,2.109091e+05
1,LGEHNLDVLEGNEQFLNAAK,24800000.0,1.240000e+06
2,LGEHNLDVLEGNEQFLNAAK,322000.0,1.610000e+04
3,LGEHNLDVLEGNEQFLNAAK,259000.0,1.295000e+04
4,LGEHNLDVLEGNEQFLNAAK,119000.0,5.950000e+03


### Exporting txt files of stripped peptides at confidence cutoffs:

In [8]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep206moddup = peaksdb206[["Peptide"]]

# keep only the stripped peptide column <1% FDR
# this is what we'll use for UniPept input, etc
pep206dup = peaksdb206[["stripped_peptide"]]

# deduplicate both of these lists
pep206mod = pep206moddup.drop_duplicates()
pep206 = pep206dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 206:", len(pep206moddup))
print("Deduplicated modified PeaksDB peptides in 206:", len(pep206mod))
print("Total stripped PeaksDB peptides in 206:", len(pep206dup))
print("Deduplicated stripped PeaksDB peptides in 206:", len(pep206))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep206.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/206_CV_T00_GD_PDB/206_CV_T00_GD_PDB_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/206_CV_T00_GD_PDB/206_CV_T00_GD_PDB_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/206_CV_T00_GD_PDB/206_CV_T00_GD_PDB_stripped_peptides.fas

# look at the stripped peptides
pep206.head()

Total modified PeaksDB peptides in 206: 99
Deduplicated modified PeaksDB peptides in 206: 99
Total stripped PeaksDB peptides in 206: 99
Deduplicated stripped PeaksDB peptides in 206: 89


,stripped_peptide
0,SCLISGWGNTK
1,LGEHNIDVLEGNEQFINAAK
5,SSGSSYPSLLQCLK
6,IITHPNFNGNTLDNDIMLIK
7,IDVLEGNEQFINAAK


## NAAF correction and exporting files with AA and PTM totals:

In [9]:
# made a new dataframe that contains the suMN of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb206['A'].sum(),
        'C': peaksdb206['C'].sum(),
        'D': peaksdb206['D'].sum(),
        'E': peaksdb206['E'].sum(),
        'F': peaksdb206['F'].sum(),
        'G': peaksdb206['G'].sum(),
        'H': peaksdb206['H'].sum(),
        'I': peaksdb206['I'].sum(),
        'K': peaksdb206['K'].sum(),
        'L': peaksdb206['L'].sum(),
        'M': peaksdb206['M'].sum(),
        'N': peaksdb206['N'].sum(),
        'P': peaksdb206['P'].sum(),
        'Q': peaksdb206['Q'].sum(),
        'R': peaksdb206['R'].sum(),
        'S': peaksdb206['S'].sum(),
        'T': peaksdb206['T'].sum(),
        'V': peaksdb206['V'].sum(),
        'W': peaksdb206['W'].sum(),
        'Y': peaksdb206['Y'].sum(),
        'c-carb': peaksdb206['c-carb'].sum(),
        'm-oxid': peaksdb206['m-oxid'].sum(),
        'n-deam': peaksdb206['n-deam'].sum(),
        'q-deam': peaksdb206['q-deam'].sum(),
        'Total area': peaksdb206['Area'].sum(),
        'Total length': peaksdb206['stripped_length'].sum()
       }

totalpeaksdb206 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                              'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                              'c-carb', 'm-oxid', 
                                              'n-deam', 'q-deam', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb206['% C w/ carb'] = totalpeaksdb206['c-carb'] / totalpeaksdb206['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb206['% M w/ oxid'] = totalpeaksdb206['m-oxid'] / totalpeaksdb206['M'] 

# calculate percentage of N's that are deamidated
totalpeaksdb206['% N w/ deam'] = totalpeaksdb206['n-deam'] / totalpeaksdb206['N'] 

# calculate percentage of N's that are deamidated
totalpeaksdb206['% Q w/ deam'] = totalpeaksdb206['q-deam'] / totalpeaksdb206['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb206['NAAF denom.'] = totalpeaksdb206['Total area'] / totalpeaksdb206['Total length']

# write modified dataframe to new txt file
totalpeaksdb206.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/206_CV_T00_GD_PDB/206_CV_T00_GD_PDB_totals.csv")

totalpeaksdb206.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,93,22,47,76,43,103,36,101,47,118,...,5,12,8,455612590.0,1141,1.0,0.5,0.096,0.177778,399309.894829


In [10]:
# use the calculated NAAF factor (in totalpeaksdb206 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 399309.894829

# we'll use -10lgP > 20 as an approximate cutoff for the <1.0% FDR which is what we actually have
peaksdb206['NAAF factor'] = (peaksdb206['NAAF_num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMN
peaksdb206_NAAF = peaksdb206[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                              'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                              'c-carb', 'm-oxid', 'n-deam', 'q-deam']].copy()

# multiply the NAAF20 factor by the AAs to normalize its abundance by peak area and peptide length

peaksdb206_NAAF['A-NAAF20'] = peaksdb206_NAAF['A'] * peaksdb206['NAAF factor']
peaksdb206_NAAF['C-NAAF20'] = peaksdb206_NAAF['C'] * peaksdb206['NAAF factor']
peaksdb206_NAAF['D-NAAF20'] = peaksdb206_NAAF['D'] * peaksdb206['NAAF factor']
peaksdb206_NAAF['E-NAAF20'] = peaksdb206_NAAF['E'] * peaksdb206['NAAF factor']
peaksdb206_NAAF['F-NAAF20'] = peaksdb206_NAAF['F'] * peaksdb206['NAAF factor']
peaksdb206_NAAF['G-NAAF20'] = peaksdb206_NAAF['G'] * peaksdb206['NAAF factor']
peaksdb206_NAAF['H-NAAF20'] = peaksdb206_NAAF['H'] * peaksdb206['NAAF factor']
peaksdb206_NAAF['I-NAAF20'] = peaksdb206_NAAF['I'] * peaksdb206['NAAF factor']
peaksdb206_NAAF['K-NAAF20'] = peaksdb206_NAAF['K'] * peaksdb206['NAAF factor']
peaksdb206_NAAF['L-NAAF20'] = peaksdb206_NAAF['L'] * peaksdb206['NAAF factor']
peaksdb206_NAAF['M-NAAF20'] = peaksdb206_NAAF['M'] * peaksdb206['NAAF factor']
peaksdb206_NAAF['N-NAAF20'] = peaksdb206_NAAF['N'] * peaksdb206['NAAF factor']
peaksdb206_NAAF['P-NAAF20'] = peaksdb206_NAAF['P'] * peaksdb206['NAAF factor']
peaksdb206_NAAF['Q-NAAF20'] = peaksdb206_NAAF['Q'] * peaksdb206['NAAF factor']
peaksdb206_NAAF['R-NAAF20'] = peaksdb206_NAAF['R'] * peaksdb206['NAAF factor']
peaksdb206_NAAF['S-NAAF20'] = peaksdb206_NAAF['S'] * peaksdb206['NAAF factor']
peaksdb206_NAAF['T-NAAF20'] = peaksdb206_NAAF['T'] * peaksdb206['NAAF factor']
peaksdb206_NAAF['V-NAAF20'] = peaksdb206_NAAF['V'] * peaksdb206['NAAF factor']
peaksdb206_NAAF['W-NAAF20'] = peaksdb206_NAAF['W'] * peaksdb206['NAAF factor']
peaksdb206_NAAF['Y-NAAF20'] = peaksdb206_NAAF['Y'] * peaksdb206['NAAF factor']

# multiply the NAAF20 factor by the PTMN normalize its abundance by peak area and peptide length

peaksdb206_NAAF['ccarb-NAAF20'] = peaksdb206_NAAF['c-carb'] * peaksdb206_NAAF['NAAF factor']
peaksdb206_NAAF['moxid-NAAF20'] = peaksdb206_NAAF['m-oxid'] * peaksdb206_NAAF['NAAF factor']
peaksdb206_NAAF['ndeam-NAAF20'] = peaksdb206_NAAF['n-deam'] * peaksdb206_NAAF['NAAF factor']
peaksdb206_NAAF['qdeam-NAAF20'] = peaksdb206_NAAF['q-deam'] * peaksdb206_NAAF['NAAF factor']


# write the dataframe to a new csv
peaksdb206_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/206_CV_T00_GD_PDB/206_CV_T00_GD_PDB_naaf.csv")

peaksdb206_NAAF.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,I,...,R-NAAF20,S-NAAF20,T-NAAF20,V-NAAF20,W-NAAF20,Y-NAAF20,ccarb-NAAF20,moxid-NAAF20,ndeam-NAAF20,qdeam-NAAF20
0,SCLISGWGNTK,0.528184,3,2,0,1,0,3,0,1,...,0.0,1.056368,1.056368,0.000000,0.528184,0.0,1.056368,0.0,0.000000,0.00000
1,LGEHNIDVLEGNEQFINAAK,3.105358,2,0,1,3,1,2,1,2,...,0.0,0.000000,0.000000,3.105358,0.000000,0.0,0.000000,0.0,0.000000,0.00000
2,LGEHNIDVLEGNEQFINAAK,0.040320,2,0,1,3,1,2,1,2,...,0.0,0.000000,0.000000,0.040320,0.000000,0.0,0.000000,0.0,0.000000,0.04032
3,LGEHNIDVLEGNEQFINAAK,0.032431,2,0,1,3,1,2,1,2,...,0.0,0.000000,0.000000,0.032431,0.000000,0.0,0.000000,0.0,0.032431,0.00000
4,LGEHNIDVLEGNEQFINAAK,0.014901,2,0,1,3,1,2,1,2,...,0.0,0.000000,0.000000,0.014901,0.000000,0.0,0.000000,0.0,0.014901,0.00000


In [11]:
# made a new dataframe that contains the suMN of NAAF normalized AAs for peaksdb206 peaksdb results
# also contains the suMN of the NAAF-corrected PTMN occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb206_NAAF['NAAF factor'].sum(),
        'A': peaksdb206_NAAF['A-NAAF20'].sum(),
        'C': peaksdb206_NAAF['C-NAAF20'].sum(),
        'D': peaksdb206_NAAF['D-NAAF20'].sum(),
        'E': peaksdb206_NAAF['E-NAAF20'].sum(),
        'F': peaksdb206_NAAF['F-NAAF20'].sum(),
        'G': peaksdb206_NAAF['G-NAAF20'].sum(),
        'H': peaksdb206_NAAF['H-NAAF20'].sum(),
        'I': peaksdb206_NAAF['I-NAAF20'].sum(),
        'K': peaksdb206_NAAF['K-NAAF20'].sum(),
        'L': peaksdb206_NAAF['L-NAAF20'].sum(),
        'M': peaksdb206_NAAF['M-NAAF20'].sum(),
        'N': peaksdb206_NAAF['N-NAAF20'].sum(),
        'P': peaksdb206_NAAF['P-NAAF20'].sum(),
        'Q': peaksdb206_NAAF['Q-NAAF20'].sum(),
        'R': peaksdb206_NAAF['R-NAAF20'].sum(),
        'S': peaksdb206_NAAF['S-NAAF20'].sum(),
        'T': peaksdb206_NAAF['T-NAAF20'].sum(),
        'V': peaksdb206_NAAF['V-NAAF20'].sum(),
        'W': peaksdb206_NAAF['W-NAAF20'].sum(),
        'Y': peaksdb206_NAAF['Y-NAAF20'].sum(),
        'c-carb': peaksdb206_NAAF['ccarb-NAAF20'].sum(),
        'm-oxid': peaksdb206_NAAF['moxid-NAAF20'].sum(),
        'n-deam': peaksdb206_NAAF['ndeam-NAAF20'].sum(),
        'q-deam': peaksdb206_NAAF['qdeam-NAAF20'].sum()
       }

totalpeaksdb206_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', \
                                                   'G', 'H', 'I','K', 'L', 'M', \
                                                   'N', 'P', 'Q', 'R', 'S', \
                                                   'T', 'V', 'W', 'Y', 'c-carb', \
                                                   'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaksdb206_NAAF['% C w/ carb.'] = totalpeaksdb206_NAAF['c-carb'] / totalpeaksdb206_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaksdb206_NAAF['% M w/ oxid'] = totalpeaksdb206_NAAF['m-oxid'] / totalpeaksdb206_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb206_NAAF['% N w/ deam'] = totalpeaksdb206_NAAF['n-deam'] / totalpeaksdb206_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb206_NAAF['% Q w/ deam'] = totalpeaksdb206_NAAF['q-deam'] / totalpeaksdb206_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb206_NAAF['NAAF check'] = totalpeaksdb206_NAAF['NAAF'] / 399309.894829

# write modified dataframe to new txt file, same name + totals
totalpeaksdb206_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/206_CV_T00_GD_PDB/206_CV_T00_GD_PDB_naaf_totals.csv")

totalpeaksdb206_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,127.720091,93.852791,2.818227,11.704124,24.197819,8.368052,23.055174,6.446947,18.463569,7.144952,...,2.246202,2.818227,1.167397,3.971374,2.930385,1.0,0.963034,0.090492,0.242732,0.00032
